In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
 def load_user_data(user_id):
    dataset_dir = os.path.join('..','dataset_processing', 'transformed_data', user_id,)
    df = pd.read_csv(dataset_dir+"/train.csv")
    orders = df.iloc[:, 2:].values
    return orders

In [4]:
def generate_input_output_pairs(user_data, sequence_length):
    input_seqs = []
    output_seqs = []
    for i in range(len(user_data) - sequence_length):
        input_seq = np.array(user_data[i:i + sequence_length])
        output_seq = np.array(user_data[i + sequence_length])
        input_seqs.append(input_seq)
        output_seqs.append(output_seq)
    return input_seqs, output_seqs

In [22]:
def build_lstm_model(input_shape, lstm_units=64, dropout_rate=0.2, num_outputs=1):
    # model = tf.keras.Sequential([
    #     tf.keras.layers.LSTM(lstm_units, input_shape=input_shape, return_sequences=True),
    #     tf.keras.layers.Dropout(dropout_rate),
    #     tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_outputs, activation='sigmoid'))
    # ])
   
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(lstm_units, input_shape=(5, 4615)),
        tf.keras.layers.Dense(4615, activation='sigmoid')
    ])
    return model

In [7]:
optimizer = tf.keras.optimizers.Adam()
def train_model(model, X_train, y_train, epochs=10, batch_size=32, validation_split=0.2):
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

In [8]:
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

In [9]:
users = os.listdir(os.path.join('..','dataset_processing', 'transformed_data'))

In [10]:
input_seq = []
output_seq = []
for user in tqdm(users, desc="Processing Users"):
    user_data = load_user_data(f"{user}")
    input, output = generate_input_output_pairs(user_data, 5)
    input_seq += input
    output_seq += output
input_arr = np.array(input_seq)
output_arr = np.array(output_seq)

Processing Users: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 831/831 [01:42<00:00,  8.12it/s]


In [11]:
input_seq = None

In [12]:
output_seq = None

In [13]:
X_train, X_test, y_train, y_test = train_test_split(input_arr, output_arr, test_size=0.2, random_state=42)

In [14]:
input_arr = output_arr = None

In [15]:
user_data = None

In [23]:
model = build_lstm_model((1,2))

In [24]:
train_model(model, X_train, y_train)

Epoch 1/10
687/687 [==============================] - 13s 10ms/step - loss: nan - accuracy: 0.0925 - val_loss: nan - val_accuracy: 0.0015
Epoch 2/10
687/687 [==============================] - 5s 7ms/step - loss: nan - accuracy: 0.0010 - val_loss: nan - val_accuracy: 0.0015
Epoch 3/10
687/687 [==============================] - 5s 7ms/step - loss: nan - accuracy: 0.0010 - val_loss: nan - val_accuracy: 0.0015
Epoch 4/10
687/687 [==============================] - 5s 7ms/step - loss: nan - accuracy: 0.0010 - val_loss: nan - val_accuracy: 0.0015
Epoch 5/10
687/687 [==============================] - 5s 7ms/step - loss: nan - accuracy: 0.0010 - val_loss: nan - val_accuracy: 0.0015
Epoch 6/10
687/687 [==============================] - 5s 7ms/step - loss: nan - accuracy: 0.0010 - val_loss: nan - val_accuracy: 0.0015
Epoch 7/10
687/687 [==============================] - 5s 7ms/step - loss: nan - accuracy: 0.0010 - val_loss: nan - val_accuracy: 0.0015
Epoch 8/10
687/687 [==========================

In [18]:
X_train.shape

(27443, 5, 4615)

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 64)                1198080   
                                                                 
 dense_2 (Dense)             (None, 4615)              299975    
                                                                 
Total params: 1,498,055
Trainable params: 1,498,055
Non-trainable params: 0
_________________________________________________________________


In [102]:
y_train.shape

(1868, 4615)

AttributeError: module 'keras.api._v2.keras.utils' has no attribute 'vis_utils'